# 🏋️ Neural Style Transfer - Model Training Notebook

This notebook demonstrates how to **train a Transformer network** for fast neural style transfer.
Unlike the original optimization-based method (Gatys et al.), this approach trains a **feedforward model** for each style.

By the end of this notebook, you will:
- Understand the training pipeline
- Learn about content/style/TV losses
- Monitor training progress with logs
- Save your own style transfer model

## 📦 Setup & Imports

In [7]:
import os
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid

from models.definitions.transformer_net import TransformerNet
from models.definitions.perceptual_loss_net import PerceptualLossNet
import utils.utils as utils

## ⚙️ Configuration Setup

In [8]:
# Paths
base_dir = os.getcwd()
training_config = {
    'style_img_name': 'psychedelic.jpg',  # must exist in data/styles/
    'content_weight': 1e0,
    'style_weight': 2e5,
    'tv_weight': 0,
    'num_of_epochs': 2,
    'subset_size': None,  # Limit data for quick test (10K) / Use None to train on entire dataset
    'enable_tensorboard': True,
    'image_log_freq': 100,
    'console_log_freq': 50,
    'checkpoint_freq': 200,
    'image_size': 256,
    'batch_size': 4,
    'dataset_path': os.path.join(base_dir, 'data', 'dataset'),
    'style_images_path': os.path.join(base_dir, 'data', 'styles'),
    'model_binaries_path': os.path.join(base_dir, 'models', 'binaries'),
    'checkpoints_path': os.path.join(base_dir, 'models', 'checkpoints', 'cubism')
}

os.makedirs(training_config['model_binaries_path'], exist_ok=True)
os.makedirs(training_config['checkpoints_path'], exist_ok=True)

## 🧩 Interactive Config: Select Style and Parameters

In [9]:
import ipywidgets as widgets
from IPython.display import display

# Style Image Selector
dropdown_style = widgets.Dropdown(
    options=sorted(os.listdir(training_config['style_images_path'])),
    value=training_config['style_img_name'],
    description='Style Image:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Epochs input
slider_epochs = widgets.IntSlider(
    value=training_config['num_of_epochs'],
    min=1, max=10, step=1,
    description='Epochs:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Style weight input
style_weight_text = widgets.FloatText(
    value=training_config['style_weight'],
    description='Style Weight:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Subset size input (None means full dataset)
subset_text = widgets.Text(
    value=str(training_config['subset_size']) if training_config['subset_size'] is not None else '',
    description='Subset Size (leave blank for all):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Display all widgets together
display(dropdown_style, slider_epochs, style_weight_text, subset_text)

Dropdown(description='Style Image:', index=6, layout=Layout(width='50%'), options=('candy.jpg', 'cubism.jpg', …

IntSlider(value=2, description='Epochs:', layout=Layout(width='50%'), max=10, min=1, style=SliderStyle(descrip…

FloatText(value=200000.0, description='Style Weight:', layout=Layout(width='50%'), style=DescriptionStyle(desc…

Text(value='', description='Subset Size (leave blank for all):', layout=Layout(width='50%'), style=TextStyle(d…

## 🧾 Final Training Configuration

In [14]:
# Update config values from user inputs
training_config['style_img_name'] = dropdown_style.value
training_config['num_of_epochs'] = slider_epochs.value
training_config['style_weight'] = style_weight_text.value

# Subset size logic
if subset_text.value.strip() == '':
    training_config['subset_size'] = None
else:
    try:
        training_config['subset_size'] = int(subset_text.value.strip())
    except ValueError:
        print("Invalid subset size. Using full dataset.")
        training_config['subset_size'] = None

# Display final training config before starting
print("🧾 Final Training Configuration:")
for k, v in training_config.items():
    print(f"{k:20s}: {v}")

🧾 Final Training Configuration:
style_img_name      : psychedelic.jpg
content_weight      : 1.0
style_weight        : 300000.0
tv_weight           : 0
num_of_epochs       : 2
subset_size         : 100
enable_tensorboard  : True
image_log_freq      : 100
console_log_freq    : 50
checkpoint_freq     : 200
image_size          : 256
batch_size          : 4
dataset_path        : /Users/chaitanyamalani/Desktop/CMN/Study/Projects/neural-style-transfer/data/dataset
style_images_path   : /Users/chaitanyamalani/Desktop/CMN/Study/Projects/neural-style-transfer/data/styles
model_binaries_path : /Users/chaitanyamalani/Desktop/CMN/Study/Projects/neural-style-transfer/models/binaries
checkpoints_path    : /Users/chaitanyamalani/Desktop/CMN/Study/Projects/neural-style-transfer/models/checkpoints/cubism


## 🧠 Initialize Model & Style

In [15]:
# Device setup
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Networks
transformer_net = TransformerNet().train().to(device)
perceptual_loss_net = PerceptualLossNet(requires_grad=False).to(device)
optimizer = Adam(transformer_net.parameters())

# Load style image
style_path = os.path.join(training_config['style_images_path'], training_config['style_img_name'])
style_img = utils.prepare_img(
    style_path, target_shape=512, device=device, batch_size=training_config['batch_size']
)

# Get style features (Gram matrices)
style_features = perceptual_loss_net(style_img)
target_grams = [utils.gram_matrix(x) for x in style_features]

/Users/chaitanyamalani/Desktop/CMN/Study/Projects/neural-style-transfer/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/chaitanyamalani/Desktop/CMN/Study/Projects/neural-style-transfer/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 🖼️ Load Dataset

In [16]:
train_loader = utils.get_training_data_loader(training_config)
print("Data loaded:", len(train_loader), "batches")

Using 200 datapoints (50 batches) for training.
Data loaded: 25 batches


## 🔁 Training Loop

In [17]:
writer = SummaryWriter()
utils.print_header(training_config)

acc_content, acc_style, acc_tv = 0, 0, 0
start_time = time.time()

for epoch in range(training_config['num_of_epochs']):
    for batch_id, (content_batch, _) in enumerate(train_loader):
        content_batch = content_batch.to(device)

        # Forward pass
        stylized_batch = transformer_net(content_batch)
        content_feats = perceptual_loss_net(content_batch)
        stylized_feats = perceptual_loss_net(stylized_batch)

        # Content loss (relu2_2)
        content_loss = training_config['content_weight'] * torch.nn.functional.mse_loss(
            stylized_feats.relu2_2, content_feats.relu2_2
        )

        # Style loss
        current_grams = [utils.gram_matrix(x) for x in stylized_feats]
        style_loss = 0
        for g1, g2 in zip(target_grams, current_grams):
            style_loss += torch.nn.functional.mse_loss(g1, g2)
        style_loss = style_loss * training_config['style_weight'] / len(target_grams)

        # Total variation (TV) loss
        tv_loss = training_config['tv_weight'] * utils.total_variation(stylized_batch)

        # Total loss
        total_loss = content_loss + style_loss + tv_loss

        # Backward
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Accumulate for logs
        acc_content += content_loss.item()
        acc_style += style_loss.item()
        acc_tv += tv_loss.item()

        global_step = epoch * len(train_loader) + batch_id

        # TensorBoard logs
        if training_config['enable_tensorboard']:
            writer.add_scalar("Loss/Content", content_loss.item(), global_step)
            writer.add_scalar("Loss/Style", style_loss.item(), global_step)
            writer.add_scalar("Loss/TV", tv_loss.item(), global_step)

            if batch_id % training_config['image_log_freq'] == 0:
                grid = make_grid(stylized_batch[:4].detach().cpu().clamp(0, 1))
                writer.add_image("Stylized", grid, global_step)

        # Console log
        if batch_id % training_config['console_log_freq'] == 0:
            elapsed = (time.time() - start_time) / 60
            print(f"Epoch {epoch+1}/{training_config['num_of_epochs']} | Batch {batch_id}/{len(train_loader)} | "
                  f"Elapsed: {elapsed:.2f} min\nContent: {acc_content:.4f} | Style: {acc_style:.4f} | TV: {acc_tv:.4f}")
            acc_content, acc_style, acc_tv = 0, 0, 0

        # Save checkpoints
        if training_config['checkpoint_freq'] and (batch_id+1) % training_config['checkpoint_freq'] == 0:
            ckpt = utils.get_training_metadata(training_config)
            ckpt['state_dict'] = transformer_net.state_dict()
            ckpt['optimizer_state'] = optimizer.state_dict()
            fname = f"ckpt_{epoch+1}_{batch_id+1}.pth"
            torch.save(ckpt, os.path.join(training_config['checkpoints_path'], fname))

Learning the style of psychedelic.jpg style image.
********************************************************************************
Hyperparams: content_weight=1.0, style_weight=300000.0, tv_weight=0
********************************************************************************
Logging to console every 50 batches.
Saving checkpoint models every 200 batches.
Tensorboard enabled.
Run "tensorboard --logdir=runs --samples_per_plugin images=50"
Open http://localhost:6006/ in your browser.
********************************************************************************
Epoch 1/2 | Batch 0/25 | Elapsed: 0.01 min
Content: 11.8847 | Style: 102.1060 | TV: 0.0000
Epoch 2/2 | Batch 0/25 | Elapsed: 0.11 min
Content: 191.8587 | Style: 17789.1855 | TV: 0.0000


## 💾 Save Final Model

In [18]:
final_model = utils.get_training_metadata(training_config)
final_model['state_dict'] = transformer_net.state_dict()
final_model['optimizer_state'] = optimizer.state_dict()
model_name = f"style_{training_config['style_img_name'].split('.')[0]}_final.pth"
torch.save(final_model, os.path.join(training_config['model_binaries_path'], model_name))
print(f"\n✅ Final model saved to: {model_name}")


✅ Final model saved to: style_psychedelic_final.pth


## 🧪 What's Next?

- Use `stylization_script.py` or the demo notebook `Image_NST_Notebook` to apply your trained model
- Try with different style images